In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from collections import Counter
from TurkishStemmer import TurkishStemmer

In [2]:
# Load stopwords
stopwords_file = '../data/stopwords-tr.txt'
with open(stopwords_file, 'r', encoding='utf-8') as f:
    stopwords_list = [line.strip() for line in f]

# Load proper nouns
proper_nouns_file = '../data/isimler.txt'
with open(proper_nouns_file, 'r', encoding='utf-8') as f:
    proper_nouns = [line.strip().lower() for line in f]
    # Extend stopwords with proper nouns
stopwords_list.extend(proper_nouns)

# Load data
file_path = '../data/pre-processed_tr.csv'
minutes_tr = pd.read_csv(file_path)
print(len(minutes_tr))

KeyboardInterrupt: 

In [ ]:
# Remove speeches with word count less than 50
minutes_tr = minutes_tr[minutes_tr['speech'].apply(lambda x: len(x.split()) >= 50)]
print(len(minutes_tr))

154116


In [ ]:
phrases_to_remove = [
    "Çok teşekkür ederim Sayın Başkanım",
    "Teşekkür ederim Sayın Başkanım",
    "Çok teşekkür ederim Sayın Başkan",
    "Sayın Başkan, teşekkür ederim",
    "Teşekkür ediyoruz",
    "Teşekkür ediyorum Sayın Başkan",
    "Teşekkür ediyorum",
    "Çok teşekkür ederim",
    "Teşekkürler Başkan",
    "Teşekkür ederim",
    "Teşekkürler",
    "Teşekkür ediyorum",
    "Buyrun",
    "Sayın milletvekilleri",
    "Kabul edilmiştir",
    "Sayın Başkan",
    "Değerli arkadaşlar",
    "…",
    "Oylama işlemi tamamlanmıştır"
]

# Create a single regular expression pattern to match all phrases
pattern = re.compile("|".join([re.escape(phrase) for phrase in phrases_to_remove]), re.IGNORECASE)

# Function to remove specified phrases using regex
def remove_phrases(text, pattern):
    return pattern.sub("", text)

minutes_tr['speech'] = minutes_tr['speech'].apply(lambda x: remove_phrases(str(x), pattern))


In [ ]:
# Initialize the stemmer
stemmer = TurkishStemmer()

# Function to preprocess text
def preprocess_text(text, stopwords_list, stemmer):
    
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'\d+', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and stem tokens
    tokens = [stemmer.stem(token.lower()) for token in tokens if token.lower() not in stopwords_list]
    
    return tokens

In [ ]:
# Preprocess all speeches
minutes_tr['tokens'] = minutes_tr['speech'].apply(lambda x: preprocess_text(x, stopwords_list, stemmer))

# Flatten the list of tokens to count document frequencies
all_tokens = [token for sublist in minutes_tr['tokens'] for token in sublist]
token_counts = Counter(all_tokens)



In [ ]:
# Remove meaningles tokens after stemming and procedural tokens
tokens_to_exclude = ["türki", "alkış", "ünç", "meç", "mi", "mı", "det", "inç", "birinç", "da", "do", "eden", "etmeyen", "etmek",
                     "unç", "nci", "kabul", "arkadaş", "istiyor", "ail", "devl", "şit", "kes", "kadar", "söz", "or", "mus", "karş",
                     "diyor", "söylüyor", "çiftç", "ülk", "den", "dan", "da", "d", "e" ,"te", "ü", "gı", "tar", "mers", "de", "tasar",
                     "sür", "buyur", "ediyor", "eder", "sayı", "fıkra", "yıl", " etmek", "mecl", "maalesef", "an", "hep", "halk",
                     "lazım", "bak", "dur", "sor", "ar", "ım", "siz", "baka", "şimt", "seç", "say", "kanu", "hakk", "hükm"]
# Filter tokens with document frequency below 10
min_doc_freq = 10
tokens_to_keep = {token for token, count in token_counts.items() if count >= min_doc_freq and token not in tokens_to_exclude}

In [ ]:
# Filter tokens in each speech
minutes_tr['tokens'] = minutes_tr['tokens'].apply(lambda tokens: [token for token in tokens if token in tokens_to_keep])


In [ ]:
lda_input = minutes_tr['tokens'].tolist()

# Example output: print the first 5 token lists to check
for tokens in lda_input[5:10]:
    print(tokens)

['büyük', 'millet', 'birleşim', 'açıyor', 'çoğunluk', 'var', 'günde', 'geçiyor', 'anayasa', 'milletvekil', 'görev', 'başlama', 'ant', 'içme', 'gerekmek', 'geçen', 'birleş', 'ant', 'iç', 'nden', 'birleş', 'ant', 'içmek', 'isteyen', 'kürsü', 'davet', 'büyük', 'millet', 'başkanlık', 'cumhuri', 'anayasa', 'madde', 'uyar', 'bakan', 'kuru', 'istifa', 'yen', 'hükümet', 'kurulunca', 'bakan', 'kuru', 'görev', 'devam', 'istendik', 'bilgi', 'sunar', 'bilgi', 'sunul', 'cumhurbaşkanlık', 'bakan', 'kuru', 'kurulma', 'görevlendirme', 'ilişk', 'tezkere', 'var', 'okutuyor', 'büyük', 'millet', 'başkanlık', 'bakan', 'kuru', 'kurulma', 'cumhuri', 'anayasa', 'madde', 'uyar', 'i̇stanbul', 'milletveki', 'adalet', 'kalkın', 'parti', 'genel', 'başkan', 'tayyip', 'erdoğan', 'görevlendiril', 'seçilecek', 'bakan', 'atanma', 'yapıldık', 'bakan', 'kuru', 'liste', 'gönderilecek', 'bilgi', 'sunar', 'günde', 'kısm', 'geçiyor', 'gündem', 'büyük', 'millet', 'başkan', 'seçim', 'yapacak', 'başkanlık', 'aday', 'göster', 'ö

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, lowercase=False)
X = vectorizer.fit_transform(lda_input)

/Users/busraalbayrak/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20 
lda = LatentDirichletAllocation(n_components=n_topics, 
                                learning_method='online', 
                                max_iter=10, 
                                random_state=0,
                                batch_size=1024,
                                n_jobs=1 
                               )

In [ ]:
# Fit the model
lda.fit(X)

# Display the top words for each topic
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
no_top_words = 20
display_topics(lda, vectorizer.get_feature_names_out(), no_top_words)

Topic 0:
yol proje ihale araç ak firma demir şehir trafik devam hav tren liman ulaştır hizmet i̇stanbul köprü havaliman hızl yap
Topic 1:
yüz ülke ekonomi oran gelir türkiye verg değer dünya sektör banka ekonomik son dış önem yatır yük şirket üzer vergi
Topic 2:
yöne merkezî bütçe hesap bütç genel kur ödenek bakanlık başkanlık müdürlük topla hizmet gelir kamu üniversite a gider b türk
Topic 3:
işçi çalışan işç dil hak çalış sendika taşeron sözleşme topl emek iş emekçi ücret yasa işveren kamu kül kadro hakk
Topic 4:
sağlık çocuk kat hastane engel sosyal kadın yaş çalışan gün hizmet şiddet insan aile hak vatandaş hastalık yüz ev bakanlık
Topic 5:
say bakan konu zaman ifade biliyor başka tane gerçek bura başkan parti söyl grup cevap yap konuşma yapıyor meclis şekil
Topic 6:
madde kanu kanun teklif değişiklik düzenle yasa düzenleme kamu komisyon hakk görev tarih şekil hükm tasarı ilişk kararname anayasa süre
Topic 7:
lira fiyat borç esnaf vatandaş çiftçi yüz destek ürün emek para üret zam 

---------
